In [1]:
import os
import shutil
import json
from PIL import Image, ImageDraw

In [2]:
DATABASE_FOLDER = '../data/wider_attribute_image'
ANNOTATION_FILE = '../data/wider_attribute_annotation/wider_attribute_trainval.json'
IMAGES_FOLDER = os.path.join(DATABASE_FOLDER, 'Image')

In [3]:
VOC_DATA = '../data/VOC0712'
VOC_ANNOTATIONS = os.path.join(VOC_DATA, 'Annotations')
VOC_IMAGESETS = os.path.join(os.path.join(VOC_DATA, 'ImageSets'), 'Main')
VOC_JPEGIMAGES = os.path.join(VOC_DATA, 'JPEGImages')

In [14]:
idx_number = 5932
numb = idx_number

In [5]:
with open(ANNOTATION_FILE) as f:
    data = json.load(f)

In [6]:
def get_gender(label):
    if (label == 11):
        return 'f'
    if (label == 1):
        return 'm'
    if (label == 0):
        return 'u'
    return 'u'

In [7]:
voc_dict = dict()

In [8]:
# формирование базы данных
for elem in data['images']: # каждый элемент - словарь
    pict = elem['file_name']
    for annotation in elem['targets']: # каждый элемент - словарь
        gend = get_gender(annotation['attribute'][0])
        bbox = list(map(int, annotation['bbox'])) 
        frame = [bbox[0]+1, bbox[1]+1, bbox[0]+bbox[2]-1, bbox[1]+bbox[3]-1]
        if gend != 'u':
            voc_content = [frame, gend]
            if not (pict in voc_dict.keys()):
                voc_dict[pict] = [voc_content]
            else:
                voc_dict[pict].append(voc_content)

In [9]:
def xml_objects(objs):
    res = ""
    for el in objs:
        res += '  <object>\n' + \
        '    <name>{}</name>\n'.format(el[1]) + \
        '    <pose>Unspecified</pose>\n' + \
        '    <truncated>0</truncated>\n' + \
        '    <difficult>0</difficult>\n' + \
        '    <bndbox>\n' + \
        '      <xmin>{}</xmin>\n'.format(el[0][0]) + \
        '      <ymin>{}</ymin>\n'.format(el[0][1]) + \
        '      <xmax>{}</xmax>\n'.format(el[0][2]) + \
        '      <ymax>{}</ymax>\n'.format(el[0][3]) + \
        '    </bndbox>\n' + \
        '  </object>\n'
    return res

In [10]:
def xml_content(file_name, voc_el):
    xml_objs = xml_objects(voc_el)
    if (xml_objs != ""):
        return '<?xml version="1.0" encoding="utf-8"?>\n' + \
                '<annotation>\n' + \
                '  <folder>VOC0712</folder>\n' + \
                '  <filename>{}</filename>\n'.format(file_name) + \
                '  <source>\n' + \
                '    <database>KAMAZ Database</database>\n' + \
                '    <annotation>KAMAZ</annotation>\n' + \
                '    <image>flickr</image>\n' + \
                '    <flickrid>0</flickrid>\n' + \
                '  </source>\n' + \
                '  <owner>\n' + \
                '    <flickrid>Unspecified</flickrid>\n' + \
                '    <name>Unspecified</name>\n' + \
                '  </owner>\n' + \
                '  <size>\n' + \
                '    <width>1920</width>\n' + \
                '    <height>1080</height>\n' + \
                '    <depth>3</depth>\n' + \
                '  </size>\n' + \
                '  <segmented>0</segmented>\n' + xml_objs + '</annotation>'

In [11]:
def create_marking_xml(file_name, voc_el):
    with open(os.path.join(VOC_ANNOTATIONS, file_name+ '.xml'), 'w') as output_file:
        str_data = xml_content(file_name + '.jpg', voc_el)
        output_file.write(str_data)

In [12]:
train_list_files = []
val_list_files = []
for k in voc_dict:
    voc_el = voc_dict[k]
    file_name = '{:0>6}'.format(idx_number)
    shutil.copyfile(os.path.join(IMAGES_FOLDER, k), os.path.join(VOC_JPEGIMAGES, file_name + '.jpg'))
    create_marking_xml(file_name, voc_el)
    if 'train/' in k:
        train_list_files.append(file_name)
    if 'val/' in k:
        val_list_files.append(file_name)
    idx_number += 1

In [30]:
with open(os.path.join(VOC_IMAGESETS, 'train_wad_imagesets.txt'), 'w') as output_file:
    for el in train_list_files:
        output_file.write(el+'\n')
with open(os.path.join(VOC_IMAGESETS, 'val_wad_imagesets.txt'), 'w') as output_file:
    for el in val_list_files:
        output_file.write(el+'\n')